# So easy, *voilà* IPyMaps!

In this example notebook, we demonstrate how voila can render custom interactive maps with built-in search. Try entering something like the following into the search field:

- `Paris` – addresses and places
- `52.5 13.4` – latitude and longitude
- `9F4MGC22+22` – Google Pluscodes (a.k.a. Open Location Codes)
- `https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json` – URLs to GeoJSON files
- `countries (a predefined GeoJSON object with the URL above)`– local names of predefined GeoJSON objects
- `osm://node["amenity"="post_box"](52.52, 13.35, 52.54, 13.45);` – OSM queries (converted to GeoJSON)
- `osm://node["amenity"="post_box"]{bounds};` – OSM queries using current bounds (use only for high zoom levels!)

In [ ]:
import re
import os
import json

import requests
import geojson
import osm2geojson
from ipywidgets import Text, HTML, Layout
from ipyleaflet import Map, GeoJSON, WidgetControl, FullScreenControl, LayersControl, ZoomControl, basemaps
from geopy.geocoders import Nominatim

In [ ]:
# fetch openlocationcode (single module) since it is not pip installable, yet
try:
    import openlocationcode as olc
except ImportError:
    url = 'https://raw.githubusercontent.com/google/open-location-code/master/python/openlocationcode.py'
    code = requests.get(url).text
    open('openlocationcode.py', 'w').write(code)
    import openlocationcode as olc

In [ ]:
def bbox(coord_list):
    "Find bounding box (lower-left and upper-right points) for a list of coordinates."

    box = []
    for i in (0, 1):
        res = sorted(coord_list, key=lambda x:x[i])
        box.append((res[0][i], res[-1][i]))
    ll, ur = [box[0][0], box[1][0]], [box[0][1], box[1][1]]
    return [ll, ur]

In [ ]:
def overpass(query: str, fmt='json'):
    """
    Run a query on OSM Overpass API and return an XML string or a JSON/GeoJSON object.
    """    
    if fmt in ['json', 'geojson']:
        data = '[out:json];' + query
    elif fmt == 'xml':
        data = '[out:xml];' + query
    else:
        raise ValueError('Format must be one of: json xml geojson')

    url = 'http://overpass-api.de/api/interpreter'
    params = dict(data=data)
    text = requests.get(url, params=params).text
    res = text
    if fmt == 'json':
        res = json.loads(text)
    if fmt == 'geojson':
        gj = osm2geojson.json2geojson(text)
        res = geojson.loads(json.dumps(gj))
    return res

In [ ]:
class SearchController:
    """
    A controller acting as a relayer between a search field and a map.
    """
    def __init__(self, a_map, a_text, geocoder=None, globs=None):
        self.geocoder = geocoder or Nominatim(user_agent="ipymaps")
        self.globs = globs or {}
        self.a_map = a_map
        self.a_text = a_text
        self.a_text.on_submit(self.text_changed)
        
    def text_changed(self, widget):
        "This is called whenever the user hits enter in the search text field."

        res = self.run_query(widget.value)
        if type(res) == tuple:
            self.a_map.center = res
        elif hasattr(res, '__geo_interface__'):
            data = res
            # jump to center of data
            ll, ur = bbox(list(geojson.utils.coords(data)))
            center = [(ur[0]+ll[0])/2, (ur[1]+ll[1])/2]
            self.a_map.center = list(reversed(center))
            name = os.path.basename(widget.value)
            self.a_map.add_layer(GeoJSON(data=data, name=name))
            
    def add_osmdata(self, query):
        "Load a GeoJSON string created from an OSM query."

        ll, ur = self.a_map.bounds
        b = ll + ur
        query = query + 'out;'
        query = query.format(bounds=b)
        return overpass(query, fmt='geojson')
        
    def add_pluscode(self, query):
        "Convert a Google Pluscode (or Open Location Code) to a lat/lon."

        if olc.isValid(query):
            res = olc.decode(query)
            lat, lon = res.latlng()
            return lat, lon
        else:
            return None, None
            
    def add_geojson(self, url):
        "Load a GeoJSON string from the given URL, local or remote."

        # load data
        if url.startswith('file://'):
            gj = open(url[7:]).read()
        else:
            gj = requests.get(url).content
        return geojson.loads(gj)
        
    def run_query(self, text):
        "Run a query as given by the ``text`` string."
        
        text = text.strip()

        # GeoJSON objects in passed namespace        
        if text in self.globs:
            obj = self.globs[text]
            if hasattr(obj, '__geo_interface__'):
                return obj
        
        # pluscodes
        try:
            if re.match('[A-Z0-9\+]', text):
                lat, lon = self.add_pluscode(text)
                if lat != None and lon != None:
                    return (lat, lon)
        except ValueError:
            pass

        # lat lon
        try:
            values = re.split('[/,;: ]', text)
            lat, lon = [v for v in values if v not in '/,;:']
            return tuple(list(map(float, (lat, lon))))
        except ValueError:
            pass

        # OSM Overpass Query
        if text.startswith('osm://'):
            gj = self.add_osmdata(text[6:])
            return gj

        # URL pointing to some GeoJSON string
        url = text
        if url.endswith('.geojson') or url.endswith('.geo.json'):
            return self.add_geojson(url)

        # address to run geolocation on
        loc = self.geocoder.geocode(text)
        if loc:
            center = loc.latitude, loc.longitude
            return center

In [ ]:
# provide a default identifer that can be used in the demo UI
url = 'https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json'
countries = geojson.loads(requests.get(url).content)

In [ ]:
bm = basemaps.OpenStreetMap['Mapnik']
m = Map(center=[50, 18], zoom=4,
        zoom_control=False,
        basemap=bm,
        layout=Layout(height='600px')
)

m.layers[0].name = bm['name']

# setup search text field and controller
search_tx = Text('')
wc = WidgetControl(widget=search_tx, position='topleft')
con = SearchController(m, search_tx, globs=globals())

# add controls
m.add_control(wc)
m.add_control(ZoomControl(position='topleft'))
m.add_control(FullScreenControl(position='topleft'))
m.add_control(LayersControl(position='topright'))

m